In [0]:
%run "./Configuration"

In [0]:
# Define query to extract entityNames from sourceSystem table:
query = f"""
  SELECT entityNames
  FROM   sourceSystem
  WHERE  sourceEntityName = 'TotesysDB'
  """

# Call query_entityNames function and load json:
entity_names = json.loads(query_entityNames(query))

In [0]:
# For each entity, create a dataFrame:
for entity in entity_names:
   # Read from SILVER path in metadata database:
   globals()[f"df_{entity}"] = spark.read.format('delta').load(SILVER + entity_silver(entity))

In [0]:
# Iterate through entities to create temp views:
for entity in entity_names:
    # Create or replace tempView with nomenclature tv_design, tv_counterparty etc:
    globals()[f'df_{entity}'].createOrReplaceTempView(f"tv_{entity}")

In [0]:
%sql
SELECT d.design_name, SUM(so.units_sold) AS total_sold
, MAX(so.unit_price) AS max_price_sold
, MIN(so.unit_price) AS min_price_sold
, CAST(AVG(so.unit_price) AS numeric(10,2)) AS average_price_sold
, CAST(SUM(so.unit_price) AS numeric(10,2)) AS total_gross_revenue
FROM tv_sales_order so
JOIN tv_design d ON so.design_id = d.design_id
GROUP BY d.design_name
ORDER BY total_gross_revenue DESC;


In [0]:
%sql
SELECT l.city, COUNT(*) as number_of_orders
FROM tv_sales_order so
JOIN tv_location l ON so.agreed_delivery_location_id = l.location_id
GROUP BY l.city
ORDER BY COUNT(*) DESC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT l.city, COUNT(*) as number_of_orders
FROM tv_sales_order so
JOIN tv_location l ON so.agreed_delivery_location_id = l.location_id
GROUP BY l.city
ORDER BY COUNT(*) ASC
LIMIT 10;